# **Fine-tuning de modelo BERT para reconhecimento de entidade nomeada NER**

## Introdução

Este notebook, utiliza-se a classe **BertForTokenClassification**, incluída no módulo [Transformers](https://github.com/huggingface/transformers) da HuggingFace. Este modelo tem o BERT como sua arquitetura base incluída uma camada de classificação de token, que possibilita fazer previsões a nível de tokens, estratégia normalmente utilizada para reconhecimento de entidades nomeadas, ao invés de sequencias.

## Preparação do Ambiente

Para utilização deste notebook é necessário ter instalado no ambiente os seguintes módulos python:

* [pandas](https://pypi.org/project/pandas/)
* [numpy](https://pypi.org/project/numpy/)
* [sklearn](https://pypi.org/project/scikit-learn/)
* [pytorch](https://pypi.org/project/torch/)
* [transformers](https://pypi.org/project/transformers/)
* [seqeval](https://pypi.org/project/seqeval/)

Por padrão o ambiente do Google Colab já possui a maior parte destes módulos instalados por padrão com exceção do transformers e do seqeval (GPU version):

In [ ]:
!pip install transformers seqeval[gpu]

Inclusão dos módulos

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertConfig, BertForTokenClassification
import matplotlib.pyplot as plt

Como os algoritmos de Deep Learning podem ser acelerados significativamente quando realizados com processamento utilizando uma GPU em vez de uma CPU, é necessário certificar-se de que ao executar este Notebook a opção por GPU esteja ativada (Verifique "Runtime" - "Change runtime type" - e defina o acelerador de hardware para "GPU").

Podemos definir o dispositivo padrão como GPU usando o seguinte código (se imprimir "cuda", significa que a GPU foi reconhecida):

In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

## Carregamento e pré-processamento de dados

O Reconhecimento de Entidades Nomeadas (NER) utiliza um esquema de anotação específico, que é definido ao nível da palavra. Um formato bastante popular utilizado para este tipo de aplicação é o [IOB-tagging](https://en.wikipedia.org/wiki/Inside%E2%80%93outside%E2%80%93beginning_(tagging)), que significa Inside-Outside-Beginning.

Neste formato cada tag indica se a palavra correspondente está **dentro**, **fora** ou no **início** de uma entidade nomeada específica. A razão para isso é que entidades nomeadas podem ser formadas por mais de uma palavra.

Por exemplo, se você tiver uma frase como "Barack Obama nasceu no Havaí", então as etiquetas correspondentes seriam [B-PERS, I-PERS, O, O, B-GEO]. B-PERS
significa que a palavra "Barack" é o início de uma pessoa, I-PERS significa que
a palavra "Obama" está dentro de uma pessoa, "O" significa que a palavra "nasceu" está fora de uma entidade nomeada, e assim por diante. Portanto, geralmente temos tantas etiquetas quanto palavras em uma frase.

Assim, ao preparar dados para treinar um modelo de Deep Learning para NER, é necessário ter seus dados neste formato IOB (ou formatos semelhantes, como [BILOU](https://stackoverflow.com/questions/17116446/what-do-the-bilou-tags-mean-in-named-entity-recognition)).

Abaixo segue indicação de algumas ferramentas que auxiliam o processo de anotação:

- [Prodigy](https://prodi.gy/)
- [Tagtog](https://docs.tagtog.com/)
- [Doccano](https://doccano.herokuapp.com/)
- [Label Studio](https://labelstud.io/)

Embora a utilização dessas ferramentas gere um arquivo estruturado, pode ser necessário realizar algum pré-processamento nos dados para ajustar a saída ao formato específico necessário para o treinamento.

In [ ]:
data = pd.read_csv("colab.csv", encoding='utf-8', sep=';')
data

Vamos verificar quantas sentenças e palavras (e suas respectivas etiquetas) existem neste conjunto de dados:

In [ ]:
data.count()

Como podemos ver, há 2.206 sentenças no dataset, totalizando mais de 20 mil palavras e etiquetas. Isso corresponde a uma média de 9 palavras por frase.

Vamos dar uma olhada nas diferentes classes e em sua frequência:

In [ ]:
print("Number of tags: {}".format(len(data.Tag.unique())))
frequencies = data.Tag.value_counts()
frequencies

Com a retirada do sufixo das etiquetas é possível chegar a seguinte distribuição:

In [ ]:
# Cria uma cópia da coluna 'Tag' column
tags = data['Tag'].copy()

# Divide as tags considerando o texto a partir do separador "-"
tags = tags.apply(lambda x: x.split('-')[1] if '-' in x else x)

# Conta as ocorrências de cada tag
frequencies = tags.value_counts()
print(frequencies)

Na base utilizada neste projeto existem poucas representações da entidade SUBESTACAO, além disso, esta entidade normalmente aparece com sobreposição em relação a entidade LINHA. Devido a isso foi necessária sua exclusão da base, visto que sua presença pode prejudicar o treinamento das demais classes.

In [ ]:
entities_to_remove = ["B-SUBESTACAO", "I-SUBESTACAO"]
data = data[~data.Tag.isin(entities_to_remove)]
data.head()

Nas próximas células é feita o ajuste para agrupar as palavras em suas sentenças:

In [ ]:
# O pandas possui uma função muito útil chamada "forward fill" para preencher valores ausentes com base no último valor não-NaN superior:
data = data.fillna(method='ffill')
data.head()

In [ ]:
# Cria uma nova coluna chamada "sentence" que agrupa as palavras por frase
data['sentence'] = data[['Sentence #','Word','Tag']].groupby(['Sentence #'])['Word'].transform(lambda x: ' '.join(x))

# Cria uma nova coluna chamada "word_labels" que agrupa as etiquetas por frase
data['word_labels'] = data[['Sentence #','Word','Tag']].groupby(['Sentence #'])['Tag'].transform(lambda x: ','.join(x))
data.head()

Para possibilitar o treinamento e interpretação dos resultados dois dicionários são criados: um que mapeia tags para índices e outro que mapeia índices para suas tags.

Isso é necessário na criação dos rótulos (já que os computadores trabalham com números (índices), em vez de palavras (etiquetas)).


In [ ]:
label2id = {k: v for v, k in enumerate(data.Tag.unique())}
id2label = {v: k for v, k in enumerate(data.Tag.unique())}
label2id

No exemplo podem ser observados 13 tags únicas.

Apenas as colunas "sentence" e "word_labels" serão utilizadas, além disso as duplicatas devem ser removidas:

In [ ]:
data = data[["sentence", "word_labels"]].drop_duplicates().reset_index(drop=True)
data.head()

In [ ]:
len(data)

Let's verify that a random sentence and its corresponding tags are correct:

In [ ]:
data.iloc[41].sentence

In [ ]:
data.iloc[41].word_labels

In [ ]:
# Distribuição de comprimento de sentenças

fig, ax = plt.subplots(figsize=(10, 5))
ax.hist([len(x.split(',')) for x in data.word_labels], bins=15)
ax.set_xlabel('Número de palavras em uma sentença')
ax.set_ylabel('Quantidade')
plt.show()


In [ ]:
# prompt: need a df showing the perncentile of data included if considered sentences that have the lenght smaller or equal than: 2,4,8,16,32,64 words

data['word_length'] = data['sentence'].apply(lambda x: len(x.split()))
data
data.word_length.describe()
threshold = [2,4,8,16,32,64]
quantiles = [data.word_length <= x for x in threshold]
for i in range(len(quantiles)):
  print('Sentenças com {} palavras ou menos: {}%'.format(threshold[i], round(quantiles[i].mean()*100, 2)))


## Definição e Treinamento do Modelo

In [ ]:
# @title Parâmetros para Treino
MAX_LEN = 64 # @param [2, 4, 8, 16, 32, 64, 128] {type:"raw"}
TRAIN_BATCH_SIZE = 16 # @param [2, 4, 8, 16, 32] {type:"raw"}
VALID_BATCH_SIZE = 16 # @param [2, 4, 8, 16, 32] {type:"raw"}
EPOCHS = 6 # @param {type:"slider", min:1, max:15, step:1}
LEARNING_RATE = 5e-05 # @param {type:"number"}
MAX_GRAD_NORM = 10 # @param {type:"integer"}
MODEL = "neuralmind/bert-base-portuguese-cased" # @param ["neuralmind/bert-base-portuguese-cased", "neuralmind/bert-large-portuguese-cased"] {allow-input: true}
tokenizer = BertTokenizer.from_pretrained(MODEL)

### Treinamento

#### **Preparing the dataset and dataloader**

Now that our data is preprocessed, we can turn it into PyTorch tensors such that we can provide it to the model. Let's start by defining some key variables that will be used later on in the training/evaluation process:

A tricky part of NER with BERT is that BERT relies on **wordpiece tokenization**, rather than word tokenization. This means that we should also define the labels at the wordpiece-level, rather than the word-level!

For example, if you have word like "Washington" which is labeled as "b-gpe", but it gets tokenized to "Wash", "##ing", "##ton", then we will have to propagate the word’s original label to all of its wordpieces: "b-gpe", "b-gpe", "b-gpe". The model should be able to produce the correct labels for each individual wordpiece. The function below (taken from [here](https://github.com/chambliss/Multilingual_NER/blob/master/python/utils/main_utils.py#L118)) implements this.






In [ ]:
def tokenize_and_preserve_labels(sentence, text_labels, tokenizer):
    """
    Word piece tokenization makes it difficult to match word labels
    back up with individual word pieces. This function tokenizes each
    word one at a time so that it is easier to preserve the correct
    label for each subword. It is, of course, a bit slower in processing
    time, but it will help our model achieve higher accuracy.
    """

    tokenized_sentence = []
    labels = []

    sentence = sentence.strip()

    for word, label in zip(sentence.split(), text_labels.split(",")):

        # Tokenize the word and count # of subwords the word is broken into
        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)

        # Add the tokenized word to the final tokenized word list
        tokenized_sentence.extend(tokenized_word)

        # Add the same label to the new list of labels `n_subwords` times
        labels.extend([label] * n_subwords)

    return tokenized_sentence, labels

Note that this is a **design decision**. You could also decide to only label the first wordpiece of each word and let the model only learn this (this is what was done in the original BERT paper, see Github discussion [here](https://github.com/huggingface/transformers/issues/64#issuecomment-443703063)). Another design decision could be to give the first wordpiece of each word the original word label, and then use the label “X” for all subsequent subwords of that word.

All of them lead to good performance.

Next, we define a regular PyTorch [dataset class](https://pytorch.org/docs/stable/data.html) (which transforms examples of a dataframe to PyTorch tensors). Here, each sentence gets tokenized, the special tokens that BERT expects are added, the tokens are padded or truncated based on the max length of the model, the attention mask is created and the labels are created based on the dictionary which we defined above.

For more information about BERT's inputs, see [here](https://huggingface.co/transformers/glossary.html).  

In [ ]:
class dataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __getitem__(self, index):
        # step 1: tokenize (and adapt corresponding labels)
        sentence = self.data.sentence[index]
        word_labels = self.data.word_labels[index]
        tokenized_sentence, labels = tokenize_and_preserve_labels(sentence, word_labels, self.tokenizer)

        # step 2: add special tokens (and corresponding labels)
        tokenized_sentence = ["[CLS]"] + tokenized_sentence + ["[SEP]"] # add special tokens
        labels.insert(0, "O") # add outside label for [CLS] token
        labels.insert(-1, "O") # add outside label for [SEP] token

        # step 3: truncating/padding
        maxlen = self.max_len

        if (len(tokenized_sentence) > maxlen):
          # truncate
          tokenized_sentence = tokenized_sentence[:maxlen]
          labels = labels[:maxlen]
        else:
          # pad
          tokenized_sentence = tokenized_sentence + ['[PAD]'for _ in range(maxlen - len(tokenized_sentence))]
          labels = labels + ["O" for _ in range(maxlen - len(labels))]

        # step 4: obtain the attention mask
        attn_mask = [1 if tok != '[PAD]' else 0 for tok in tokenized_sentence]

        # step 5: convert tokens to input ids
        ids = self.tokenizer.convert_tokens_to_ids(tokenized_sentence)

        label_ids = [label2id[label] for label in labels]
        # the following line is deprecated
        #label_ids = [label if label != 0 else -100 for label in label_ids]

        return {
              'ids': torch.tensor(ids, dtype=torch.long),
              'mask': torch.tensor(attn_mask, dtype=torch.long),
              #'token_type_ids': torch.tensor(token_ids, dtype=torch.long),
              'targets': torch.tensor(label_ids, dtype=torch.long)
        }

    def __len__(self):
        return self.len

Now, based on the class we defined above, we can create 2 datasets, one for training and one for testing. Let's use a 80/20 split:

In [ ]:
train_size = 0.8
train_dataset = data.sample(frac=train_size,random_state=200)
test_dataset = data.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)

print("FULL Dataset: {}".format(data.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = dataset(train_dataset, tokenizer, MAX_LEN)
testing_set = dataset(test_dataset, tokenizer, MAX_LEN)

Let's have a look at the first training example:

In [ ]:
training_set[0]

Let's verify that the input ids and corresponding targets are correct:

In [ ]:
training_set[0]["ids"]

In [ ]:
# print the first 30 tokens and corresponding labels
for token, label in zip(tokenizer.convert_ids_to_tokens(training_set[0]["ids"][:30]), training_set[0]["targets"][:30]):
  print('{0:10}  {1}'.format(token, id2label[label.item()]))

Now, let's define the corresponding PyTorch dataloaders:

In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

#### **Defining the model**

Here we define the model, BertForTokenClassification, and load it with the pretrained weights of "bert-base-uncased". The only thing we need to additionally specify is the number of labels (as this will determine the architecture of the classification head).

Note that only the base layers are initialized with the pretrained weights. The token classification head of top has just randomly initialized weights, which we will train, together with the pretrained weights, using our labelled dataset. This is also printed as a warning when you run the code cell below.

Then, we move the model to the GPU.

In [ ]:
model = BertForTokenClassification.from_pretrained(MODEL,
                                                   num_labels=len(id2label),
                                                   id2label=id2label,
                                                   label2id=label2id)
model.to(device)

#### **Training the model**

Before training the model, let's perform a sanity check, which I learned thanks to Andrej Karpathy's wonderful [cs231n course](http://cs231n.stanford.edu/) at Stanford (see also his [blog post about debugging neural networks](http://karpathy.github.io/2019/04/25/recipe/)). The initial loss of your model should be close to -ln(1/number of classes) = -ln(1/17) = 2.83.

Why? Because we are using cross entropy loss. The cross entropy loss is defined as -ln(probability score of the model for the correct class). In the beginning, the weights are random, so the probability distribution for all of the classes for a given token will be uniform, meaning that the probability for the correct class will be near 1/17. The loss for a given token will thus be -ln(1/17). As PyTorch's [CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html) (which is used by `BertForTokenClassification`) uses *mean reduction* by default, it will compute the mean loss for each of the tokens in the sequence (in other words, for all of the 512 tokens). The mean of 512 times -log(1/17) is, you guessed it, -log(1/17).  

Let's verify this:



In [ ]:
ids = training_set[0]["ids"].unsqueeze(0)
mask = training_set[0]["mask"].unsqueeze(0)
targets = training_set[0]["targets"].unsqueeze(0)
ids = ids.to(device)
mask = mask.to(device)
targets = targets.to(device)
outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
initial_loss = outputs[0]
initial_loss

This looks good. Let's also verify that the logits of the neural network have a shape of (batch_size, sequence_length, num_labels):

In [ ]:
tr_logits = outputs[1]
tr_logits.shape

Next, we define the optimizer. Here, we are just going to use Adam with a default learning rate. One can also decide to use more advanced ones such as AdamW (Adam with weight decay fix), which is [included](https://huggingface.co/transformers/main_classes/optimizer_schedules.html) in the Transformers repository, and a learning rate scheduler, but we are not going to do that here.

In [ ]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

Now let's define a regular PyTorch training function. It is partly based on [a really good repository about multilingual NER](https://github.com/chambliss/Multilingual_NER/blob/master/python/utils/main_utils.py#L344).

In [ ]:
# Defining the training function on the 80% of the dataset for tuning the bert model
def train(epoch):
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()

    for idx, batch in enumerate(training_loader):

        ids = batch['ids'].to(device, dtype = torch.long)
        mask = batch['mask'].to(device, dtype = torch.long)
        targets = batch['targets'].to(device, dtype = torch.long)

        outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
        loss, tr_logits = outputs.loss, outputs.logits
        tr_loss += loss.item()

        nb_tr_steps += 1
        nb_tr_examples += targets.size(0)

        if idx % 100==0:
            loss_step = tr_loss/nb_tr_steps
            print(f"Training loss per 100 training steps: {loss_step}")

        # compute training accuracy
        flattened_targets = targets.view(-1) # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        # now, use mask to determine where we should compare predictions with targets (includes [CLS] and [SEP] token predictions)
        active_accuracy = mask.view(-1) == 1 # active accuracy is also of shape (batch_size * seq_len,)
        targets = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)

        tr_preds.extend(predictions)
        tr_labels.extend(targets)

        tmp_tr_accuracy = accuracy_score(targets.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy

        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=MAX_GRAD_NORM
        )

        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {tr_accuracy}")

And let's train the model!

In [ ]:
for epoch in range(EPOCHS):
    print(f"Training epoch: {epoch + 1}")
    train(epoch)

## Resultados

#### **Evaluating the model**

Now that we've trained our model, we can evaluate its performance on the held-out test set (which is 20% of the data). Note that here, no gradient updates are performed, the model just outputs its logits.

In [ ]:
def valid(model, testing_loader):
    # put model in evaluation mode
    model.eval()

    eval_loss, eval_accuracy = 0, 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []

    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):

            ids = batch['ids'].to(device, dtype = torch.long)
            mask = batch['mask'].to(device, dtype = torch.long)
            targets = batch['targets'].to(device, dtype = torch.long)

            outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
            loss, eval_logits = outputs.loss, outputs.logits

            eval_loss += loss.item()

            nb_eval_steps += 1
            nb_eval_examples += targets.size(0)

            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Validation loss per 100 evaluation steps: {loss_step}")

            # compute evaluation accuracy
            flattened_targets = targets.view(-1) # shape (batch_size * seq_len,)
            active_logits = eval_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            # now, use mask to determine where we should compare predictions with targets (includes [CLS] and [SEP] token predictions)
            active_accuracy = mask.view(-1) == 1 # active accuracy is also of shape (batch_size * seq_len,)
            targets = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)

            eval_labels.extend(targets)
            eval_preds.extend(predictions)

            tmp_eval_accuracy = accuracy_score(targets.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy

    #print(eval_labels)
    #print(eval_preds)

    labels = [id2label[id.item()] for id in eval_labels]
    predictions = [id2label[id.item()] for id in eval_preds]

    #print(labels)
    #print(predictions)

    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    print(f"Validation Loss: {eval_loss}")
    print(f"Validation Accuracy: {eval_accuracy}")

    return labels, predictions

As we can see below, performance is quite good! Accuracy on the test test is > 93%.

In [ ]:
labels, predictions = valid(model, testing_loader)

However, the accuracy metric is misleading, as a lot of labels are "outside" (O), even after omitting predictions on the [PAD] tokens. What is important is looking at the precision, recall and f1-score of the individual tags. For this, we use the seqeval Python library:

### Saída

In [ ]:
from seqeval.metrics import classification_report

print(classification_report([labels], [predictions]))

print("MODEL:",MODEL)
print("MAX_LEN:",MAX_LEN)
print("TRAIN_BATCH_SIZE:",TRAIN_BATCH_SIZE)
print("VALID_BATCH_SIZE:",VALID_BATCH_SIZE)
print("EPOCHS:",EPOCHS)
print("LEARNING_RATE:",LEARNING_RATE)
print("MAX_GRAD_NORM:",MAX_GRAD_NORM)

## Inferência

In [ ]:
sentence = "inspeção detalhada na torre 37 da lt adrianópolis cachoeira paulista, isolador quebrado na fase lateral esquerda"

inputs = tokenizer(sentence, padding='max_length', truncation=True, max_length=MAX_LEN, return_tensors="pt")

# move to gpu
ids = inputs["input_ids"].to(device)
mask = inputs["attention_mask"].to(device)
# forward pass
outputs = model(ids, mask)
logits = outputs[0]

active_logits = logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size*seq_len,) - predictions at the token level

tokens = tokenizer.convert_ids_to_tokens(ids.squeeze().tolist())
token_predictions = [id2label[i] for i in flattened_predictions.cpu().numpy()]
wp_preds = list(zip(tokens, token_predictions)) # list of tuples. Each tuple = (wordpiece, prediction)

word_level_predictions = []
for pair in wp_preds:
  if (pair[0].startswith(" ##")) or (pair[0] in ['[CLS]', '[SEP]', '[PAD]']):
    # skip prediction
    continue
  else:
    word_level_predictions.append(pair[1])

# we join tokens, if they are not special ones
str_rep = " ".join([t[0] for t in wp_preds if t[0] not in ['[CLS]', '[SEP]', '[PAD]']]).replace(" ##", "")
print(str_rep)
print(word_level_predictions)

In [ ]:
from transformers import pipeline

pipe = pipeline(task="token-classification", model=model.to("cpu"), tokenizer=tokenizer, aggregation_strategy="simple")
# pipe("inspeção detalhada na torre 37 da ibntpr1, isolador quebrado na fase lateral esquerda")
pipe("cordoalha rompida no para-raios esquerdo da torre 233 na linha itumbiara porto colombia 1")

## **Salvando o Modelo**

In [ ]:
model.save_pretrained('base_m3')

In [ ]:
tokenizer.save_pretrained('base_m3')